In [14]:
from dotenv import load_dotenv

import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("No OpenAI API key found.")
else:
    print("OpenAI API key found.")

OpenAI API key found.


### Adding Tool to further instruct GPT

In [53]:
# Bag prices
bag_prices = {
    "Rico Black": "S$150",
    "Rico Red": "S$150",
    "Mini Klaki": "S$80",
    "Disney Series": "S$250"
}

def get_bag_price(bag_name):
    normalize_bag_price = {key.lower(): value for key, value in bag_prices.items()}
    return normalize_bag_price.get(bag_name.lower(), "Unknown")

price_function = {
    "name": "get_bag_price",
    "description": "Get the price of a bag. Call this whenever you need to know the bag price, for example when a customer asks 'How much is a price of Mini Klaki'",
    "parameters": {
        "type": "object",
        "properties": {
            "bag_name": {
                "type": "string",
                "description": "The price of the bag customer would like to know",
            },
        },
        "required": ["bag_name"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": price_function}]

In [58]:
import json
from openai import OpenAI

openai = OpenAI()

system_message_dict = {
    "role": "system",
    "content": "You are a helpful assistant for an Ecommerce Website who sell bags called Sift & Pick. "
               "Give short, courteous answers, no more than 1 sentence. "
               "Always be accurate. If you don't know the answer, say so."
}

def message_gpt(message, history):
    global system_message_dict
    
    messages = [system_message_dict] + history + [{"role": "user", "content": message}]
    print("Constructed messages:", json.dumps(messages, indent=2))
    
    try:
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)
        if response.choices[0].finish_reason=="tool_calls":
            tool_message  = response.choices[0].message
            tool_response, bag_name = handle_tool_call(tool_message)
            messages.append(tool_message)
            messages.append(tool_response)
            response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
            return response.choices[0].message.content
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error: {e}")
        return f"An error occurred: {e}"
    
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    print("Tool call arguments:", arguments)
    bag_name = arguments["bag_name"]
    price = get_bag_price(bag_name)
    print(f"Bag name: {bag_name}, Price: {price}")
    response = {
        "role": "tool",
        "content": json.dumps({"bag_name": bag_name,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, bag_name


### Build a Chatbot

In [59]:
import gradio as gr

gr.ChatInterface(fn=message_gpt, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.
